In [1]:
import ee
import geemap
from bqplot import pyplot as plt
from bqplot import Bars

In [2]:
landsatMap = geemap.Map()

# PREFIRE

In [3]:
point = ee.Geometry.Point([-123.090,39.87])

landsat_prefire_image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2020-07-30', '2020-08-1')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[2-7]')
)

print(ee.Date(landsat_prefire_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()) 
# landsat_prefire_image = landsat_prefire_image.select('SR_B.').multiply(0.0000275).add(-0.2)

vis_params = {'min':0, 'max':20000, 'bands': ['SR_B7', 'SR_B5', 'SR_B4']} # swir2, nir, red

landsatMap.centerObject(point, 9)
landsatMap.addLayer(landsat_prefire_image, vis_params, "Prefire")
# landsatMap

prefire_nbr = landsat_prefire_image.expression(
    '(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': landsat_prefire_image.select('SR_B5'),
        'SWIR': landsat_prefire_image.select('SR_B7'),
    })

# Define a map centered on California
# prefire_nbr_map = folium.Map(location=[40,-123], zoom_start=10)
landsatMap.centerObject(point, 9)

# Add the image layer to the map and display it.
landsatMap.add_ee_layer(prefire_nbr, {
    # 'min': -1,
    # 'max': 1,
    'palette': ['#d7191c', '#fdae61','#ffffbf', '#a6d96a','#1a9641']
}, 'prefire_nbr_map')

# landsatMap

2020-07-31


# POSTFIRE

In [4]:
point = ee.Geometry.Point([-123.090,39.87])

landsat_postfire_image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2020-10-18', '2020-10-20')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[2-7]')
)

print(ee.Date(landsat_postfire_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()) 
# landsat_postfire_image = landsat_postfire_image.select('SR_B.').multiply(0.0000275).add(-0.2)

vis_params = {'min':0, 'max':20000, 'bands': ['SR_B7', 'SR_B5', 'SR_B4']} # swir2, nir, red
landsatMap.addLayer(landsat_postfire_image, vis_params, "Postfire")
# landsatMap

postfire_nbr = landsat_postfire_image.expression(
    '(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': landsat_postfire_image.select('SR_B5'),
        'SWIR': landsat_postfire_image.select('SR_B7'),
    })

# Define a map centered the california
# postfire_nbr_map = folium.Map(location=[40,-123], zoom_start=10)
landsatMap.centerObject(point, 9)

# Add the image layer to the map and display it.
landsatMap.add_ee_layer(postfire_nbr, {
    # 'min': -1,
    # 'max': 1,
    'palette': ['#d7191c', '#fdae61','#ffffbf', '#a6d96a','#1a9641']
}, 'postfire_nbr_map')

landsatMap.addLayerControl()
landsatMap

2020-10-19


Map(center=[39.87, -123.09], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

# dNBR

In [5]:
d_nbr = prefire_nbr.subtract(postfire_nbr)

# Define a map 
landsatMap.add_ee_layer(d_nbr, {
#     'min': -0.5 ,
#     'max': 1.3,
    'palette': ['#1a9641', '#a6d96a', '#ffffbf','#fdae61','#d7191c']
}, 'd_nbr_map')


landsatMap.addLayerControl()
landsatMap

Map(center=[39.87, -123.09], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [6]:
d_nbr = d_nbr.rename('dNBR')

In [7]:
d_nbr.getInfo()

{'type': 'Image',
 'bands': [{'id': 'dNBR',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7671, 7861],
   'crs': 'EPSG:32610',
   'crs_transform': [30, 0, 404985, 0, -30, 4582215]}]}

In [8]:
# polygon = landsatMap.draw_features
polygon = ee.Geometry(d_nbr.geometry())

In [9]:
# Compute the histogram of the NIR band.  The mean and variance are only FYI.
histogram = d_nbr.reduceRegion(
    **{
        'reducer': ee.Reducer.histogram(1024, 0.015625),
        'geometry': polygon,
        'scale': 100,
        'bestEffort': False,
    }
)
hist_dict = histogram.getInfo()
print(hist_dict)

{'dNBR': {'bucketMeans': [-0.6038221940140313, -0.5859375, -0.563979217611866, -0.5513996850435323, -0.5390625, -0.5190750273278282, -0.5078125, -0.4998285376781811, -0.47511290616042606, -0.46231451185131905, -0.44719210752032024, -0.43266145272550804, -0.41701625876650605, -0.39795327387988977, -0.38322073368343196, -0.36836007406234517, -0.35050530881239417, -0.3350906259005378, -0.3193376845993256, -0.30399326225914414, -0.2883221595133364, -0.2732445851741208, -0.25704223035151313, -0.24176000840536718, -0.22662621862170845, -0.21077348519018305, -0.1956177758534487, -0.1794694544252667, -0.1637194403844727, -0.148066116000071, -0.13261626750912905, -0.11682144194441811, -0.10126760351126902, -0.08499157496688398, -0.06903459828522379, -0.05312836435590477, -0.03665699059015586, -0.022090712638299904, -0.008111875396676193, 0.0073902415875556635, 0.023093922438153322, 0.038779909571383775, 0.05445899287535458, 0.07006309874702911, 0.0856995095404007, 0.10126640647345235, 0.1169010

In [10]:
# def normalize_list(y):
#     maxi = max(y)
#     mini = min(y)
#     for i in range(len(y)):
#         y[i] = (y[i]-mini)/(maxi-mini)

In [11]:
x = hist_dict['dNBR']['bucketMeans']
y = hist_dict['dNBR']['histogram']

# normalize_list(y)

plt.bar(x, y)
plt.show()

In [12]:
# Return the DN that maximizes interclass variance in B5 (in the region).
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

    # Compute between sum of squares, where each mean partitions the data.

    def func_xxx(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = (
            aMeans.multiply(aCounts)
            .reduce(ee.Reducer.sum(), [0])
            .get([0])
            .divide(aCount)
        )
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
            bCount.multiply(bMean.subtract(mean).pow(2))
        )

    bss = indices.map(func_xxx)

    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])


threshold = otsu(histogram.get('dNBR'))
print('threshold', threshold.getInfo())

threshold 0.11690105461644164


In [13]:
classA = d_nbr.gt(threshold) # lt is less_than, gt is greater_than
landsatMap.addLayer(classA.mask(classA), {'palette': 'orange'}, 'fire')
landsatMap

Map(center=[39.87, -123.09], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [14]:
# def extract_water(image):
#     histogram = image.select('N').reduceRegion(
#         **{
#             'reducer': ee.Reducer.histogram(255, 2),
#             'geometry': polygon,
#             'scale': 10,
#             'bestEffort': True,
#         }
#     )
#     threshold = otsu(histogram.get('N'))
#     water = image.select('N').lt(threshold).selfMask()
#     return water.set({"threshold": threshold})

In [15]:
# water_images = collection.map(extract_water)

In [16]:
# Map.addLayer(water_images.first(), {"palette": "blue"}, "first water")

In [17]:
# water_images.aggregate_array("threshold").getInfo()

In [18]:
# Map